In [1]:
from utils import utils, xgb_fn, bayesian_encoding

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

from matplotlib import pyplot as plt
%matplotlib inline

from jupyterthemes import jtplot
jtplot.reset()

In [53]:
train, test = utils.load_data()
train = train.id
test = test.id

Successfully loaded train and test data


In [2]:
combined = pd.read_csv('data/combine.csv')

In [14]:
combined_dummies = combined.drop(combined.columns[combined.columns.str.startswith('ps_calc')], axis=1)
combined_dummies = combined_dummies.drop(['ps_ind_10_bin','ps_ind_11_bin','ps_ind_12_bin','ps_ind_13_bin',
                'ps_ind_18_bin','ps_car_03_cat','ps_car_10_cat'], axis=1)
combined_dummies = pd.get_dummies(combined_dummies, columns=combined_dummies.columns[combined_dummies.columns.str.endswith('cat')==True])
combined_dummies = combined_dummies.drop(['id', 'target'], axis=1)
combined_dummies = combined_dummies.fillna(-1)

In [18]:
combined_dummies = StandardScaler().fit_transform(combined_dummies)

In [32]:
combined = combined.drop(combined.columns[combined.columns.str.startswith('ps_calc')], axis=1)
combined = combined.drop(['ps_ind_10_bin','ps_ind_11_bin','ps_ind_12_bin','ps_ind_13_bin',
                'ps_ind_18_bin','ps_car_03_cat','ps_car_10_cat'], axis=1)

In [244]:
# combined_cat = combined[combined.columns[combined.columns.str.endswith('cat')]]
combined_bin = combined[combined.columns[combined.columns.str.endswith('bin')]]

In [52]:
combined = combined[combined.columns[~combined.columns.str.endswith('cat')]]
combined = combined[combined.columns[~combined.columns.str.endswith('bin')]]

In [110]:
ps_reg_03 = (combined.ps_reg_03**2)*16*100
ps_car_15 = combined.ps_car_15**2
ps_car_14 = (combined.ps_car_14**2)*10000
ps_car_13 = combined.ps_car_13**2*100000
ps_car_13 = ps_car_13.round(1)
ps_car_12 = combined.ps_car_12**2*100
ps_car_12 = ps_car_12.round(0)*100

In [202]:
kmeans = KMeans(n_clusters=10, random_state=123, n_jobs=-1)

In [208]:
combined_cat['ps_reg_03_cat'] = kmeans.fit_predict(ps_reg_03.fillna(-1).reshape(-1, 1))
combined_cat['ps_car_13_cat'] = kmeans.fit_predict(ps_car_13.fillna(-1).reshape(-1, 1))
combined_cat['ps_car_14_cat'] = kmeans.fit_predict(ps_car_14.fillna(-1).reshape(-1, 1))
combined_cat['ps_car_12_cat'] = ps_car_12
combined_cat['ps_car_15_cat'] = ps_car_15

In [223]:
combined_cat['ps_reg_01_cat'] = combined.ps_reg_01*10
combined_cat['ps_reg_02_cat'] = combined.ps_reg_02*10

In [225]:
combined_cat = pd.concat([combined_cat,combined[['ps_ind_01', 'ps_ind_03', 'ps_ind_14', 'ps_ind_15']]], axis=1)

In [232]:
combined_cat_bin = pd.get_dummies(combined_cat, dummy_na=True, columns=combined_cat.columns)

In [245]:
combined_bin = pd.concat([combined[['id', 'target']], combined_bin,combined_cat_bin], axis=1)

In [249]:
combined_bin.iloc[train,:].to_csv('data/bin_train.csv', index=False)
combined_bin.iloc[test,:].to_csv('data/bin_test.csv', index=False)

In [250]:
combined_cat.iloc[train,:].to_csv('data/cat_train.csv', index=False)
combined_cat.iloc[test,:].to_csv('data/cat_test.csv', index=False)

In [48]:
combined_percent = (combined.iloc[:,2:].rank(axis=0)/combined.iloc[:,2:].rank(axis=0).max())

In [49]:
combined_percent = pd.concat([combined[['id','target']], combined_percent], axis=1)

In [55]:
combined_percent.iloc[train,:].to_csv('data/percent_train.csv', index=False)
combined_percent.iloc[test,:].to_csv('data/percent_test.csv', index=False)